In [ ]:
#instala as bibliotecas necessárias para a execução
!pip install pandas
!pip install seaborn
!pip install numpy
!pip install matplotlib
!pip install sklearn

In [ ]:
#importando as bibliotecas que serão utilizadas
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, recall_score, precision_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
import warnings

#serão ignoradas todas mensagens de warning (aviso) que aparecer no notebook
warnings.filterwarnings('ignore')

## Abrindo o arquivo com os dados

In [ ]:
arquivo_xls = pd.ExcelFile('data/teste_smarkio_Lbs.xls')
dados_analise_ml = pd.read_excel(arquivo_xls, 'Análise_ML')
dados_nlp = pd.read_excel(arquivo_xls, 'NLP')

In [ ]:
#esses dados serão utilizados para resolver as atividades de 1 a 4.
dados_analise_ml.head(5)

In [ ]:
#esses dados serão utilizados para resolver a atividade 5.
dados_nlp.head(5)

## Atividades

### 1- Análise exploratória dos dados utilizando estatística descritiva e inferencial,considerando uma, duas e/ou mais variáveis.

In [ ]:
#pred_class = classe que foi identificada pelo modelo
#probabilidade = a probabilidade da classe que o modelo identificou
#status = status da classificação de acordo com um especialista
#true_class = classe verdadeira. Se for nula, assumir o valor do pred_class
#se pred_class = true_class, então o modelo acertou a classe

In [ ]:
#nós temos um total de 643 dados (também podemos chamar de linhas) e 4 colunas
dados_analise_ml.shape

In [ ]:
#primeiramente iremos analisar os tipos das colunas que temos
#'pred_class', 'probabilidade' e 'true_class' são colunas numéricas
#'status' é do tipo object. Ou seja, se trata de uma coluna categórica
dados_analise_ml.info()

In [ ]:
dados_analise_ml.describe(include='all') #iremos incluir todas as colunas, até mesmo as categóricas (status)

#a única coluna que possui valores nulos é a 'true_class', elas serão tratadas posteriormente
#a coluna 'status' só possui dois valores possíveis

#### Análise da coluna Status

In [ ]:
#analisando a coluna 'status'

fig = plt.figure(figsize=(6, 4))
plt.title('Análise da coluna Status')
sns.countplot(dados_analise_ml['status'])
fig.savefig('images/analise_status.png')

#a coluna 'status' possui dois tipos de valores: approved e revision
#approved é o valor que aparece com mais frequência (600), o que representa, aproximadamente, 93% do total
#revision aparece somente 43 vezes
#isso indica que temos mais dados aprovados do que os que precisam de uma revisão

#### Análise da coluna True_class

In [ ]:
#analisando a coluna 'true_class'
dados_analise_ml_semnull = dados_analise_ml.copy() #armazenando uma cópia do nosso dataframe em outra variável

#iremos fazer uma iteração em cada linha do nosso dataframe
for index, linha in dados_analise_ml_semnull.iterrows():
    
    #verificamos se o valor para 'True_class' é nan (nulo)
    if np.isnan(linha['True_class']):
        
        #se for nulo, iremos armazenar o valor do 'Pred_class' no seu lugar
        dados_analise_ml_semnull.loc[index, 'True_class'] = linha['Pred_class']

In [ ]:
#verificando se ainda existe algum valor nulo no dataframe e se a inserção funcionou
dados_analise_ml_semnull.isnull().sum()

In [ ]:
#verificando se inserimos os valores certos

#será criado um dataframe temporário para fazermos a comparação
#teremos duas colunas 'True_class': a da esquerda é antes de ser tratada e a direita depois de ser tratada.
#veremos que os valores foram inseridos corretamente
temp_data = pd.concat([dados_analise_ml, dados_analise_ml_semnull['True_class']], axis=1)
temp_data.tail(30)

In [ ]:
#deleta a variável temporária já que não usaremos ela para mais nada
del temp_data

In [ ]:
#usaremos só os dados onde o status é 'approved', pois foram confirmados e aprovados pelos cientistas
dados_analise_ml_semnull_approved = dados_analise_ml_semnull[dados_analise_ml_semnull['status'] == 'approved'].copy()

fig = plt.figure(figsize=(18, 12))
plt.title('Análise da coluna True_class (Approved)')
sns.countplot(y=dados_analise_ml_semnull_approved['True_class'], 
              orient='h',
              order=dados_analise_ml_semnull_approved['True_class'].value_counts().index)
fig.savefig('images/analise_true_class.png')

In [ ]:
#analisando o histograma da coluna 'True_class'
fig = plt.figure(figsize=(7, 4))
plt.title('Histograma da coluna True_class (Approved)')
sns.histplot(dados_analise_ml_semnull_approved['True_class'], kde=True)
fig.savefig('images/histograma_true_class.png')

#através do gráfico anterior (countplot) e do histograma mostrado abaixo
#podemos perceber que as classes não são uniformemente distribuída, pois temos
#mais classes 0, 2, 3, 4, 60, 74 e 77 do que as demais. Principalmente a 74.

#podemos perceber também que os dados tem uma forma parecida com a de 
# dados que são assimétrico para a esquerda (positivo)

In [ ]:
#usaremos só os dados onde o status é 'revision'
dados_analise_ml_semnull_revision = dados_analise_ml_semnull[dados_analise_ml_semnull['status'] == 'revision'].copy()

fig = plt.figure(figsize=(18, 12))
plt.title('Análise da coluna True_class (Revision)')
sns.countplot(y=dados_analise_ml_semnull_revision['True_class'], 
              orient='h', 
              order=dados_analise_ml_semnull_revision['True_class'].value_counts().index)
fig.savefig('images/analise_true_class_revision.png')

#o compartamento dos dados com status 'revision' são parecidos com os dados
#que possuem status 'approved'

In [ ]:
#analisando o histograma da coluna 'True_class' (revision)
fig = plt.figure(figsize=(7, 4))
plt.title('Histograma da coluna True_class (Revision)')
sns.histplot(dados_analise_ml_semnull_revision['True_class'], kde=True)
fig.savefig('images/histograma_true_class_revision.png')

In [ ]:
#temos um total de 69 classes diferentes (Approved)
dados_analise_ml_semnull_approved['True_class'].nunique()

In [ ]:
#temos um total de 73 classes diferentes (Total)
dados_analise_ml_semnull['True_class'].nunique()

In [ ]:
#analisando o boxplot da coluna, verificando se existe outlier
fig = plt.figure(figsize=(6, 4))
plt.title('Análise boxplot True_class (Approved)')
sns.boxplot(dados_analise_ml_semnull_approved['True_class'])
fig.savefig('images/boxplot_true_class.png')

#### Análise da coluna Pred_class

In [ ]:
#analisando a coluna 'pred_class'
dados_analise_ml_semnull_approved = dados_analise_ml_semnull[dados_analise_ml_semnull['status'] == 'approved'].copy()

fig = plt.figure(figsize=(18, 12))
plt.title('Análise da coluna Pred_class (Approved)')
sns.countplot(y=dados_analise_ml_semnull_approved['Pred_class'],
              orient='h',
              order=dados_analise_ml_semnull_approved['Pred_class'].value_counts().index)
fig.savefig('images/analise_pred_class.png')

#as classes que mais apareceram na coluna 'Pred_class', que são os valores previstos pelo modelo
#também são as que mais apareceram na coluna 'True_class', que são: 2, 3, 4, 60, 74 e 77.
#o único que não aparece é a classe 0.
#isso pode indicar que o valor 0, que está presente na coluna 'True_class', não representa uma classe
#e pode ter sido um erro de digitação ou que o modelo tem dificuldades para reconhece-la

In [ ]:
#analisando o histograma da coluna 'Pred_class'
fig = plt.figure(figsize=(7, 4))
plt.title('Histograma da coluna Pred_class (Approved)')
sns.histplot(dados_analise_ml_semnull_approved['Pred_class'], kde=True)
fig.savefig('images/histograma_pred_class.png')

In [ ]:
#quantidade de classes diferentes previstas quando o status é approved
dados_analise_ml_semnull_approved['Pred_class'].nunique()

In [ ]:
#quantidade de classes diferentes previstas no total
dados_analise_ml_semnull['Pred_class'].nunique()

In [ ]:
#criando uma nova coluna no nosso dataframe para verificarmos se o algoritmo acertou a classe
#se o classe real = prevista, colocamos o valor 1. Se não, o valor 0.
dados_analise_ml_semnull_approved['Acertou'] = np.where(dados_analise_ml_semnull_approved['Pred_class'] == dados_analise_ml_semnull_approved['True_class'],
                                                        1, 0)

In [ ]:
dados_analise_ml_semnull_approved.head(10)

In [ ]:
#analisando as classes que tiveram o maior número de acertos
dados_analise_ml_semnull_approved_acertou = dados_analise_ml_semnull_approved[dados_analise_ml_semnull_approved['Acertou'] == 1]

fig = plt.figure(figsize=(18, 12))
plt.title('Análise das colunas que tiveram um maior acerto')
sns.countplot(y=dados_analise_ml_semnull_approved_acertou['True_class'],
              orient='h',
              order=dados_analise_ml_semnull_approved_acertou['True_class'].value_counts().index)
fig.savefig('images/analise_classes_maior_acerto.png')

In [ ]:
#analisando as classes que tiveram o maior número de erros
dados_analise_ml_semnull_approved_errou = dados_analise_ml_semnull_approved[dados_analise_ml_semnull_approved['Acertou'] == 0]

fig = plt.figure(figsize=(18, 12))
plt.title('Análise das colunas que tiveram mais erros')
sns.countplot(y=dados_analise_ml_semnull_approved_errou['True_class'],
              orient='h',
              order=dados_analise_ml_semnull_approved_errou['True_class'].value_counts().index)
fig.savefig('images/analise_classes_maior_erro.png')

In [ ]:
#analisando o boxplot da coluna, verificando se existe outlier
fig = plt.figure(figsize=(6, 4))
plt.title('Análise boxplot Pred_class (Approved)')
sns.boxplot(dados_analise_ml_semnull_approved['Pred_class'])
fig.savefig('images/boxplot_pred_class.png')

#### Análise da coluna Probabilidade

In [ ]:
#analisando a coluna probabilidade

fig = plt.figure(figsize=(6, 4))
plt.title('Análise da coluna de probabilidade (Approved)')
sns.histplot(dados_analise_ml_semnull_approved['probabilidade'], kde=True)
fig.savefig('images/histograma_probabilidade.png')

In [ ]:
#analisando a coluna probabilidade quando acerta a classe
fig = plt.figure(figsize=(6, 4))
plt.title('Análise da coluna de probabilidade quando acerta a classe')
sns.histplot(dados_analise_ml_semnull_approved_acertou['probabilidade'], kde=True)
fig.savefig('images/histograma_probabilidade_acertou.png')

In [ ]:
#correlação entre as colunas
fig = plt.figure(figsize=(6, 4))
sns.heatmap(dados_analise_ml_semnull_approved.corr(), annot=True)
fig.savefig('images/correlacao_colunas.png')

#podemos ver que a coluna probabilidade tem uma correlação relativamente alta com a coluna 'acertou'

### 2- Calcule o desempenho do modelo de classificação utilizando pelo menos três métricas.

In [ ]:
#Por se tratar de um problema de classificação, as métricas utilizadas são voltadas para
#esse tipo de problema.
#Um problema de classificação consiste em atribuir uma classe, dentre todas as possibilidades possíveis,
#para uma determinada entrada.

#### Confusion Matrix

In [ ]:
confusion_matrix = confusion_matrix(dados_analise_ml_semnull['True_class'],
                                    dados_analise_ml_semnull['Pred_class'])

fig = plt.figure(figsize=(24, 18))
sns.heatmap(confusion_matrix, cmap='Blues')
plt.xlabel('Pred_class')
plt.ylabel('True_class')

#a confusion matrix (matriz de confusão) é boa para podermos visualizar os falsos negativos e positivos
#das classes.
#as áreas mais escuras quer dizer que teve uma maior ocorrência. 
#podemos observar que existem algumas áreas escuras fora da diagonal principal da matriz
#nosso modelo confunde bastante o 0 com o 28 e o 44 com o 0.

#### Accuracy Score

In [ ]:
score = accuracy_score(dados_analise_ml_semnull_approved['True_class'],
                       dados_analise_ml_semnull_approved['Pred_class'])

score

#### Recall Score

In [ ]:
#calcula o score considerando os falsos negativo e verdadeiro positivo
score = recall_score(dados_analise_ml_semnull_approved['True_class'],
                     dados_analise_ml_semnull_approved['Pred_class'],
                     average='weighted')

score

#### Precision Score

In [ ]:
#calcula o score considerando os falsos positivos e verdadeiro positivo
score = precision_score(dados_analise_ml_semnull_approved['True_class'],
                        dados_analise_ml_semnull_approved['Pred_class'],
                        average='weighted')

score

### 3- Crie um classificador que tenha como output se os dados com status igual a revision estão corretos ou não (Sugestão : Técnica de cross-validation K-fold).

In [ ]:
X_train = dados_analise_ml_semnull_approved['Pred_class']
y_train = dados_analise_ml_semnull_approved['True_class']

#pega somente as linhas onde o status é 'revision'
dados_analise_ml_revision = dados_analise_ml_semnull[dados_analise_ml_semnull['status'] == 'revision'].copy()

X_test = dados_analise_ml_revision['Pred_class']
y_test = dados_analise_ml_revision['True_class']

In [ ]:
from sklearn.ensemble import RandomForestClassifier

#para que possamos treinar o nosso classificador, primeiro devemos converter nossos
#valores para um array de numpy.
#isso deverá ser feito para que os nossos valores tenham a dimensão correta.
#deverá ser feito tanto para o X_train quanto o X_test
X_train_array = np.array(X_train)
X_train_array = X_train_array.reshape(-1, 1)

X_test_array = np.array(X_test)
X_test_array = X_test_array.reshape(-1, 1)

random_forest = RandomForestClassifier(random_state=23)

#treinando o modelo com os dados que possuem status 'approved'
random_forest.fit(X_train_array, y_train)

#testando o modelo com os dados que possuem status 'revision'
pred = random_forest.predict(X_test_array)
pred

In [ ]:
#criando um novo dataframe para podermos visualizar melhor a classe prevista e a classe real
dataframe_comparacao = pd.DataFrame({
    'Classe_real': y_test,
    'Classe_prevista': pred
})

#criando uma nova coluna no nosso dataframe para verificarmos se o algoritmo acertou a classe
#se o classe real = prevista, colocamos o valor 1. Se não, o valor 0.
dataframe_comparacao['Acertou'] = np.where(dataframe_comparacao['Classe_real'] == dataframe_comparacao['Classe_prevista'],
                                          1, 0)

dataframe_comparacao

In [ ]:
#podemos ver que o modelo acertou uma grande quantidade de dados que estavam com status 'revision'
print("Total de acertos: " + str(dataframe_comparacao['Acertou'].sum()) + " de " + str(dataframe_comparacao.shape[0]))
print("Porcentagem: " + str(dataframe_comparacao['Acertou'].sum()/dataframe_comparacao.shape[0]))

#iremos obter um score de, aproximadamente, 0.814 no classificador criado
#isso poderia ser melhorado usando outros algoritmos mais eficientes (por exemplo: XGBoost e LightGBM)
#e também usando técnicas para melhorar os hiper parâmetros do modelo (GridSearch, RandomSearch ou Optuna)

### 4- Compare três métricas de avaliação aplicadas ao modelo e descreva sobre a diferença.

#### Accuracy Score

In [ ]:
#calcula o score utilizando verdadeiro positivo, verdadeiro negativo, falso positivo e falso negativo
#fórmula= (vp + vn) / (vp + vn + fp + fn)
#vp= verdadeiro positivo
#vn= verdadeiro negativo
#fp= falso positivo
#fn= falso negativo

score = accuracy_score(dataframe_comparacao['Classe_real'],
                       dataframe_comparacao['Classe_prevista'])

score

#### Recall Score

In [ ]:
#calcula o score utilizando verdadeiro positivo e falso negativo
#fórmula= vp / (vp + fn)
#vp= verdadeiro positivo
#fn= falso negativo

#estamos usando o 'weighted' para calcular o score de cada classe e aplicando um peso sobre ele
#no nosso caso, como as classes não estão distribuídas de forma uniforme é necessário usar o peso
#para que o score fique balanceado
score = recall_score(dataframe_comparacao['Classe_real'],
                     dataframe_comparacao['Classe_prevista'],
                     average='weighted')

score

In [ ]:
#utilizando o 'macro' só iremos calcular o score de classe e não aplicamos um peso sobre ela
#podemos notar a diferença de um score para o outro
score = recall_score(dataframe_comparacao['Classe_real'],
                     dataframe_comparacao['Classe_prevista'],
                     average='macro')

score

#### Precision Score

In [ ]:
#calcula o score utilizando verdadeiro positivo e falso positivo
#fórmula= vp / (vp + fp)
#vp= verdadeiro positivo
#fp= falso positivo

#estamos usando o 'weighted' para calcular o score de cada classe e aplicando um peso sobre ele
#no nosso caso, como as classes não estão distribuídas de forma uniforme é necessário usar o peso
#para que o score fique balanceado
score = precision_score(dataframe_comparacao['Classe_real'],
                        dataframe_comparacao['Classe_prevista'],
                        average='weighted')

score

In [ ]:
#utilizando o 'macro' só iremos calcular o score de classe e não aplicamos um peso sobre ela
#podemos notar a diferença de um score para o outro
score = precision_score(dataframe_comparacao['Classe_real'],
                     dataframe_comparacao['Classe_prevista'],
                     average='macro')

score

### 5- Crie um classificador, a partir da segunda aba - NLP do arquivo de dados, quepermita identificar qual trecho de música corresponde às respectivas artistas listadas (Sugestão: Naive Bayes Classifier).

In [ ]:
#temos 518 dados e duas colunas
#colunas: letra e artista
#letra: trecho da música
#artista: cantora referente ao trecho da música (letra)
dados_nlp.shape

In [ ]:
#verificando se existe linhas com nulo
dados_nlp.describe()

In [ ]:
fig = plt.figure(figsize=(7, 4))
plt.title('Análise dos Artistas')
sns.countplot(dados_nlp['artista'])

#Temos dois artistas: Beyoncé e Rihanna
#Beyoncé aparece 274 vezes, aproximadamente 53% 
#Rihanna aparece 244 vezes, aproximadamente 47%

#### Criação do classificador

In [ ]:
#separando os dados em X(features) e y(target)

X = dados_nlp['letra']
y = dados_nlp['artista']

#separando os dados para treinamento do classificador e para testá-lo
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=23)

#para a extração de features será utilizado o método bags of words
#o que esse método faz é associar um número inteiro para cada palavra
#e depois armazenará a quantidade de vezes que essa palavra apareceu.
#a função CountVectorizer já realiza todo esse processo de pré processamento e 
#tokenização (associar ao número inteiro) das palavras.
cv = CountVectorizer(stop_words='english')

X_train_tratado = cv.fit_transform(X_train)
X_test_tratado = cv.transform(X_test)

naive_bayes = MultinomialNB()

#treinando o classificador
naive_bayes.fit(X_train_tratado, y_train)

#testando o classificador com os dados de teste
pred = naive_bayes.predict(X_test_tratado)
np.mean(pred == y_test)

#iremos obter um score de 0.75 no classificador criado
#isso poderia ser melhorado usando outros algoritmos mais eficientes (por exemplo o SVM), já que o Naive Bayes é mais básico
#e também usando técnicas para melhorar os hiper parâmetros do modelo (GridSearch, RandomSearch ou Optuna)